# Crawling AFL Match Player data from FootyWire.com #
## For Brownlow Predictor Project ##

This notebook contains not only functions which scrape data into dataframes which are then stored as .csvs, but also functions that process the data in two different ways

Creator:`Lang (Ron) Chen` 2021 Dec - 2022

**0. Import Libraries**

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random
from urllib.parse import urljoin
import os

**1. Scrape Function**

In [2]:
def scrape(url):
    
    # Scrape first page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    section = soup.find(id='contentpagecell')
    results = section.findNext('table')
    rows = results.find_all('tr')
    
    
    
    # Check that this is a Home and Away game rather than finals. If not then just stop the process
    findround = section.findNext('table').findNext('table').findNext('table')
    if not bool(re.search(r'Round', str(findround))):
        return False, False, False    # because our normal return returns three objects
    
    
    
    # Get Brownlow votes data (but input is later)
    brownlow = list() #Mechanism to prevent non-assignment because some pages don't have bronwlow data
    for i in range(len(rows)):
        if re.search(r'Brownlow Votes:', str(rows[i])):
            brownlow = rows[i]
    if brownlow:
        players = brownlow.find_all('a')
        brownlow = list()
        for player in players:
            brownlow.append(re.findall(r'>. .+<', str(player))[0].strip('<>'))
    
    
    
    # Get raw player statistics
    results = soup.find(id='matchscoretable').findNext('table')
    rows=results.find_all('tr')
   
    tick = 0
    for i in range(len(rows)):
        if not tick and len(rows[i].find_all('tr'))>24: # From experiment, blocks that contain player data (what we want) has more than 25 rows
            team1stats=rows[i]
            tick=1
        elif len(rows[i].find_all('tr'))>24:
            team2stats=rows[i]
            break
            
    team1playerstats = team1stats.find_all('tr')[2].find_all('tr') # From experimenting with scraped blob
    team2playerstats = team2stats.find_all('tr')[2].find_all('tr')

    gamestatscol1 = getdata(team1playerstats)
    gamestatscol2 = getdata(team2playerstats)
    
    
    
    # Get advanced player statistics
    time.sleep(random.uniform(0.5, 5))    # First sleep for a random amount of time - trying to hide crawler activity
    
    urlAdv = url + '&advv=Y' # Because advanced statistic's URL is only different from orig URL by this string
    pageAdv = requests.get(urlAdv)
    soupAdv = BeautifulSoup(pageAdv.text, 'html.parser')
    
    resultsAdv = soupAdv.find(id='matchscoretable').findNext('table')
    rowsAdv=resultsAdv.find_all('tr')

    tick = 0
    for i in range(len(rowsAdv)):
        if not tick and len(rowsAdv[i].find_all('tr'))>24:
            team1stats=rowsAdv[i]
            tick=1
        elif len(rowsAdv[i].find_all('tr'))>24:
            team2stats=rowsAdv[i]
            break  
    
    team1playerstats = team1stats.find_all('tr')[2].find_all('tr')
    team2playerstats = team2stats.find_all('tr')[2].find_all('tr')
    
    gamestatscol1A = getadvdata(team1playerstats)
    gamestatscol2A = getadvdata(team2playerstats)
    
    
    
    # Append the advanced player statistics to standard player statistics
    origattb1 = list()
    for i in range(len(gamestatscol1)):
        origattb1.append(gamestatscol1[i][0])

    for i in range(len(gamestatscol1A)):
        if gamestatscol1A[i][0] not in origattb1:
            gamestatscol1.append(gamestatscol1A[i])
            
            
    origattb2 = list()
    for i in range(len(gamestatscol2)):
        origattb2.append(gamestatscol2[i][0])

    for i in range(len(gamestatscol2A)):
        if gamestatscol2A[i][0] not in origattb2:
            gamestatscol2.append(gamestatscol2A[i])
    
    
    
    # Find Winloss data
    winloss = section.findNext('table').findNext('table').findNext('table') 
    keyword = re.findall(r'>\n.*\n<', str(winloss.find_all('td')[0]))[0].strip('>\n\n<')

    v1 = int()
    v2 = int()

    if 'defeats' in keyword:
        v1 = 1
        v2 = 0
    elif 'defeated by' in keyword:
        v1 = 0
        v2 = 1
    else:
        v1 = 0.5
        v2 = 0.5
    
    winloss1 = ['Winloss']
    winloss2 = ['Winloss']
    for i in range(1, len(gamestatscol1[0])):
        winloss1.append(v1)
    for i in range(1, len(gamestatscol2[0])):
        winloss2.append(v2)
    
    gamestatscol1.append(winloss1)
    gamestatscol2.append(winloss2)
    
    

    # Collect some statistics for naming the file such as year, round, and team names (collectively metadata)
    Year = re.findall(r'\d{4},', str(winloss))[0].strip(',')
    Round = re.findall(r'Round \d+', str(winloss))[0]
    
    team1 = keyword.split()[0]
    # Hardcoded some of the two worded team names
    if team1 == 'Gold':
        team1 = 'GoldCoast'
    
    elif team1 == 'North':
        team1 = 'NorthMelbourne'
    
    elif team1 == 'Port':
        team1 = 'PortAdelaide'
    
    elif team1 == 'St':
        team1 = 'StKilda'
    
    elif team1 == 'West':
        team1 = 'WestCoast'
    
    elif team1 == 'Western':
        team1 = 'WesternBulldogs'
    
    
    team2 = keyword.split()[-1]
    if team2 == 'Coast': 
        if keyword.split()[-2] == 'Gold':
            team2 = 'GoldCoast'
        elif keyword.split()[-2] == 'West':
            team2 = 'WestCoast'
    
    elif team2 == 'Melbourne' and keyword.split()[-2] == 'North':
        team2 = 'NorthMelbourne'
    
    elif team2 == 'Adelaide' and keyword.split()[-2] == 'Port':
        team2 = 'PortAdelaide'
    
    elif team2 == 'Kilda':
        team2 = 'StKilda'
        
    elif team2 == 'Bulldogs':
        team2 = 'WesternBulldogs'

    
    
    # Add Brownlow Data into our dataframe as a column
    team1playerlist = gamestatscol1[0]
    team2playerlist = gamestatscol2[0]
    
    if brownlow:
        brownlow[0] = shorten_surname(brownlow[0])
        brownlow[1] = shorten_surname(brownlow[1])
        brownlow[2] = shorten_surname(brownlow[2])
        
        brownlowdict = {brownlow[0]: 3, brownlow[1]:2, brownlow[2]: 1}
    else:
        brownlowdict = dict()
    
    brownlowteam1 = list()
    brownlowteam1.append('Brownlow Votes')
    for i in range(1, len(team1playerlist)):
        if nametransf(team1playerlist[i]) in brownlowdict:
            brownlowteam1.append(brownlowdict[nametransf(team1playerlist[i])])
        else:
            brownlowteam1.append(0)
    gamestatscol1.append(brownlowteam1)

    brownlowteam2 = list()
    brownlowteam2.append('Brownlow Votes')
    for i in range(1, len(team2playerlist)):
        if nametransf(team2playerlist[i]) in brownlowdict:
            brownlowteam2.append(brownlowdict[nametransf(team2playerlist[i])])
        else:
            brownlowteam2.append(0)
    gamestatscol2.append(brownlowteam2)
    
    # Print games where there is an error in Brownlow Votes (i.e. Total brownlow votes does not add up to 1+2+3 = 6)
    if (sum(brownlowteam1[1:]) + sum(brownlowteam2[1:])) != 6:
        print(f'{Year} {Round} {team1} v {team2}: {sum(brownlowteam1[1:]) + sum(brownlowteam2[1:])}')
        if brownlow:
            print(brownlowdict)
        print('\n')
    
    
    return gamestatscol1, gamestatscol2, (Year, Round, team1, team2)

In [3]:
def shorten_surname(name):
    """ Function to change compound surname into semi-acronym format to fit the player name data on FootyWire.com """
    
    if '-' in name.split()[1]:
        surname_lst = name.split()[1].split('-')
        new_surname = f'{surname_lst[0][0]}-{surname_lst[1]}'
        return f'{name.split()[0]} {new_surname}'
    
    return name

In [4]:
def getdata(teamplayerstats):
    """ Function which manipulates scraped data and puts them into a suitable list format for further wrangling"""
    
    gamestatsrow = list()
    for row in teamplayerstats:
        record = list()
        cells = row.find_all('td')
        for cell in cells:
            record.append(re.findall(r'>.*<', str(cell))[0].strip('><'))
        gamestatsrow.append(record)

        
    # Add player name    
    gamestatscol = list()
    tmp = list()
    for i in range(len(gamestatsrow)):
        if i == 0:
            tmp.append(gamestatsrow[0][0])
        else:
            tmp.append(re.findall(r'>.*<',gamestatsrow[i][0])[0].strip('><'))
    gamestatscol.append(tmp)

    
    # Add other stats
    for i in range(1,16):    # Hardcoded
        tmp = list()
        for j in range(len(gamestatsrow)):
            if j == 0:
                tmp.append(re.findall(r'title=".*"', str(gamestatsrow[0][i]))[0].strip('title=""'))
            else:
                tmp.append(float(gamestatsrow[j][i]))
        gamestatscol.append(tmp)
    
    
    return gamestatscol

In [5]:
def getadvdata(teamplayerstats):
    """ Function which manipulates scraped advanced data and puts them into a suitable list format for further wrangling"""
    
    gamestatsrow = list()
    for row in teamplayerstats:
        record = list()
        cells=row.find_all('td')
        for cell in cells:
            record.append(re.findall(r'>.*<', str(cell))[0].strip('><'))
        gamestatsrow.append(record)
    
    
    # Add player name 
    gamestatscolA = list()
    tmp = list()
    for i in range(len(gamestatsrow)):
        if i == 0:
            tmp.append(gamestatsrow[0][0])
        else:
            tmp.append(re.findall(r'>.*<',gamestatsrow[i][0])[0].strip('><'))
    gamestatscolA.append(tmp)

    
    # Add other stats
    for i in range(1,18):    # Hardcoded
        tmp = list()
        for j in range(len(gamestatsrow)):
            if j == 0:
                tmp.append(re.findall(r'title=".*"', str(gamestatsrow[0][i]))[0].strip('title=""'))
            else:
                tmp.append(float(gamestatsrow[j][i]))
        gamestatscolA.append(tmp)

        
    return gamestatscolA

In [6]:
def nametransf(name):
    """ For matching full player name format of original data to 'initials + last name' format of Brownlow data """
    
    # Special case: sydney player Josh Kennedy who is recoreded as Josh P. Kennedy 
    if name == 'Josh P. Kennedy':
        return 'J Kennedy'
    
    tmp = name.split(' ')
    first_name = tmp[0]
    last_name = str()
    for i in range(1,len(tmp)):
        last_name += tmp[i]
        last_name += ' '
        
    return f'{first_name[0]} {last_name[:-1]}'

**2. Functions for saving output as CSV**

In [7]:
def combine(lst1, lst2):
    """ Function to join the two team's lists into one so we could put into dataframe and save """
    
    out = list()
    for i in range(len(lst1)):
        out.append(list())
        for j in range(len(lst1[i])):
            out[i].append(lst1[i][j])
    for i in range(len(lst1)):
        for j in range(1,len(lst2[i])):
            out[i].append(lst2[i][j])
    return out

In [8]:
def save(lst, metadata, datatype):
    """ Function to save data in form of lists as a dataframe and then a CSV """
    
    DICT = {'O': 'OriginalData', 'S': 'StandardisedData', 'M': 'ManipulatedData'}
    df = pd.DataFrame({lst[0][0]: lst[0][1:len(lst[0])]})
    for i in range(1, len(lst)):
        df.insert(column = lst[i][0], value = lst[i][1:len(lst[0])], loc = i)
    
    if not os.path.exists(f'./Data (validation fix)/{DICT[datatype]}'):
            os.mkdir(f'./Data (validation fix)/{DICT[datatype]}')
    
    df.to_csv(f'./Data/{DICT[datatype]}/{metadata[0]} {metadata[1]} {metadata[2]} v {metadata[3]} ({datatype}).csv', index = False)

<b>3. Functions for standardising Data</b>

In [9]:
def ownteamstandardise(statlist):
    """ Function for standardising data based on own team """
    
    maxval = max(statlist[1:len(statlist)])
    minval = min(statlist[1:len(statlist)]) #但到底设 0 or minval?
    
    out = list()
    out.append(f'{statlist[0]} OT')
    
    for i in range(1, len(statlist)):
        if maxval-minval == 0:
            out.append(0)
        else:
            out.append((int(statlist[i])-minval)/(maxval-minval))
    
    return out

In [10]:
def ownteamstandardise_inv(statlist):
    """ Function for standardising data based on own team (but in inverse: least = best) """
    
    maxval = max(statlist[1:len(statlist)])
    minval = min(statlist[1:len(statlist)]) #但到底设 0 or minval?
    
    out = list()
    out.append(f'{statlist[0]} OT')
    
    for i in range(1, len(statlist)):
        if maxval-minval == 0:
            out.append(0)
        else:
            out.append((int(statlist[i])-minval)/(maxval-minval))
    
    return out

In [11]:
def bothteamstandardise(statlist1, statlist2):
    """ Function for standardising data based on both teams """
    
    statlist = statlist1.copy()
    for i in range(1, len(statlist2)):
        statlist.append(statlist2[i])
    
    maxval = max(statlist[1:len(statlist)])
    minval = min(statlist[1:len(statlist)]) #但到底设 0 or minval?
    
    out = list()
    out.append(f'{statlist[0]} BT')
    
    for i in range(1, len(statlist)):
        if maxval-minval == 0:
            out.append(0)
        else:
            out.append((int(statlist[i])-minval)/(maxval-minval))
    
    out1 = out[0:len(statlist1)]
    out2 = list()
    out2.append(out[0])
    for i in range(len(statlist1), len(out)):
        out2.append(out[i])
    
    return out1, out2

In [12]:
def bothteamstandardise_inv(statlist1, statlist2):
    """ Function for standardising data based on both teams (but in inverse: least = best) """    
    
    statlist = statlist1.copy()
    for i in range(1, len(statlist2)):
        statlist.append(statlist2[i])
    
    maxval = max(statlist[1:len(statlist)])
    minval = min(statlist[1:len(statlist)]) #但到底设 0 or minval?
    
    out = list()
    out.append(f'{statlist[0]} BT')
    
    for i in range(1, len(statlist)):
        if maxval-minval == 0:
            out.append(0)
        else:
            out.append((int(statlist[i])-minval)/(maxval-minval))
    
    out1 = out[0:len(statlist1)]
    out2 = list()
    out2.append(out[0])
    for i in range(len(statlist1), len(out)):
        out2.append(out[i])
    
    return out1, out2

In [13]:
def standardise(lst1, lst2):
    """ Function which takes the two team's data as seperate lists and performs relevent standardisation based on requirements """
    
    # Use lists to specify which statistic are to be standardised and in what way
    
    #原封不动
    Orig = ['Player', 'Winloss', 'Brownlow Votes']

    #正向
    normal = ['Kicks', 'Handballs', 'Disposals', 'Marks', 'Goals', 'Behinds', 'Tackles', 'Hitouts', 'Goal Assists', 'Inside 50s', 
                   'Clearances', 'Rebound 50s', 'Frees For', 'Contested Possessions', 'Uncontested Possessions', 
                   'Effective Disposals', 'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces', 'Centre Clearances', 
                   'Stoppage Clearances', 'Score Involvements', 'Metres Gained', 'Intercepts', 'Tackles Inside 50', 'Time On Ground %', 'Disposal Efficiency %']

    #反向
    invert = ['Clangers', 'Turnovers', 'Frees Agains']
    
    out1 = list()
    out2 = list()
    
    for i in range(len(lst1)):
        if lst1[i][0] in Orig:
            out1.append(lst1[i])
            out2.append(lst2[i])
            
        elif lst1[i][0] in normal:
            out1.append(ownteamstandardise(lst1[i]))
            out2.append(ownteamstandardise(lst2[i]))
            tmp1, tmp2 = bothteamstandardise(lst1[i], lst2[i])
            out1.append(tmp1)
            out2.append(tmp2)
            
        elif lst1[i][0] in invert:
            out1.append(ownteamstandardise_inv(lst1[i]))
            out2.append(ownteamstandardise_inv(lst2[i]))
            tmp1, tmp2 = bothteamstandardise_inv(lst1[i], lst2[i])
            out1.append(tmp1)
            out2.append(tmp2)
            
            
    return out1, out2 

<b>4. Manipulation</b>

In [14]:
def ownteamrank(statlist):
    """ Function to find the rank of a player in a stat in terms of their own team """
    
    out = list()
    out.append(f'{statlist[0]} OTR')
    
    tmp = list()
    
    for i in range(1, len(statlist)): 
        tmp.append((int(statlist[i]),i))
    
    tmp.sort(reverse=True)
    
    for i in range(1, len(statlist)):
        out.append(0)
    
    switch = 0
    for i in range(len(tmp)):
        if i+1 == len(tmp):
            if switch == 1:
                value = (start+i)*0.5
                for j in range(start, i+1):
                    out[tmp[j][1]] = value+1
            else:
                out[tmp[i][1]] = i+1
                
        else:
            if switch == 1:
                if tmp[i+1][0] != tmp[i][0]:
                    switch = 0
                    value = (start+i)*0.5
                    
                    for j in range(start, i+1):
                        out[tmp[j][1]] = value+1

            else:
                if tmp[i+1][0] == tmp[i][0]:
                    start = i
                    switch = 1
                else:
                    out[tmp[i][1]] = i+1
            
            
    return out

In [15]:
def ownteamper(statlist):
    """ Function to find the quantile of a player in a stat in terms of their own team """
    
    out = list()
    out.append(f'{statlist[0]} OT%')
    total = 0
    for i in range(1, len(statlist)):
        total += int(statlist[i])
    for i in range(1, len(statlist)):
        if total == 0:
            out.append(0)
        else:
            out.append(int(statlist[i])/total)
        
    return out

In [16]:
def bothteamrank(statlist1, statlist2):
    """ Function to find the rank of a player in a stat in terms of both teams """
    
    statlist = statlist1.copy()
    for i in range(1, len(statlist2)):
        statlist.append(statlist2[i])
    
    out = list()
    out.append(f'{statlist[0]} BTR')
    
    tmp = list()
    
    for i in range(1, len(statlist)): 
        tmp.append((int(statlist[i]),i))
    
    tmp.sort(reverse=True)
    
    for i in range(1, len(statlist)):
        out.append(0)
    
    switch = 0
    for i in range(len(tmp)):
        if i+1 == len(tmp):
            if switch == 1:
                value = (start+i)*0.5
                for j in range(start, i+1):
                    out[tmp[j][1]] = value+1
            else:
                out[tmp[i][1]] = i+1
                
        else:
            if switch == 1:
                if tmp[i+1][0] != tmp[i][0]:
                    switch = 0
                    value = (start+i)*0.5
                    
                    for j in range(start, i+1):
                        out[tmp[j][1]] = value+1

            else:
                if tmp[i+1][0] == tmp[i][0]:
                    start = i
                    switch = 1
                else:
                    out[tmp[i][1]] = i+1
    
    out1 = out[0:len(statlist1)]
    out2 = list()
    out2.append(out[0])
    for i in range(len(statlist1), len(out)):
        out2.append(out[i])
            
    return out1, out2

In [17]:
def bothteamper(statlist1, statlist2):
    """ Function to find the quantile of a player in a stat in terms of both teams """
    
    statlist = statlist1.copy()
    for i in range(1, len(statlist2)):
        statlist.append(statlist2[i])
        
    out = list()
    out.append(f'{statlist[0]} BT%')
    total = 0
    for i in range(1, len(statlist)):
        total += int(statlist[i])
    for i in range(1, len(statlist)):
        if total == 0:
            out.append(0)
        else:
            out.append(int(statlist[i])/total)
    
    out1 = out[0:len(statlist1)]
    out2 = list()
    out2.append(out[0])
    for i in range(len(statlist1), len(out)):
        out2.append(out[i])
    
    return out1, out2

In [18]:
def ownteamrank_inv(statlist):
    """ Function to find the rank of a player in a stat in terms of their own team (but in inverse: least = best) """
    
    out = list()
    out.append(f'{statlist[0]} OTR')
    
    tmp = list()
    
    for i in range(1, len(statlist)): 
        tmp.append((int(statlist[i]),i))
    
    tmp.sort()
    
    for i in range(1, len(statlist)):
        out.append(0)
    
    switch = 0
    for i in range(len(tmp)):
        if i+1 == len(tmp):
            if switch == 1:
                value = (start+i)*0.5
                for j in range(start, i+1):
                    out[tmp[j][1]] = value+1
            else:
                out[tmp[i][1]] = i+1
                
        else:
            if switch == 1:
                if tmp[i+1][0] != tmp[i][0]:
                    switch = 0
                    value = (start+i)*0.5
                    
                    for j in range(start, i+1):
                        out[tmp[j][1]] = value+1

            else:
                if tmp[i+1][0] == tmp[i][0]:
                    start = i
                    switch = 1
                else:
                    out[tmp[i][1]] = i+1
            
            
    return out

In [19]:
def bothteamrank_inv(statlist1, statlist2):
    """ Function to find the rank of a player in a stat in terms of both teams (but in inverse: least = best) """
    
    statlist = statlist1.copy()
    for i in range(1, len(statlist2)):
        statlist.append(statlist2[i])
    
    out = list()
    out.append(f'{statlist[0]} BTR')
    
    tmp = list()
    
    for i in range(1, len(statlist)): 
        tmp.append((int(statlist[i]),i))
    
    tmp.sort()
    
    for i in range(1, len(statlist)):
        out.append(0)
    
    switch = 0
    for i in range(len(tmp)):
        if i+1 == len(tmp):
            if switch == 1:
                value = (start+i)*0.5
                for j in range(start, i+1):
                    out[tmp[j][1]] = value+1
            else:
                out[tmp[i][1]] = i+1
                
        else:
            if switch == 1:
                if tmp[i+1][0] != tmp[i][0]:
                    switch = 0
                    value = (start+i)*0.5
                    
                    for j in range(start, i+1):
                        out[tmp[j][1]] = value+1

            else:
                if tmp[i+1][0] == tmp[i][0]:
                    start = i
                    switch = 1
                else:
                    out[tmp[i][1]] = i+1
    
    out1 = out[0:len(statlist1)]
    out2 = list()
    out2.append(out[0])
    for i in range(len(statlist1), len(out)):
        out2.append(out[i])
            
    return out1, out2

In [20]:
def PercentageAndRank(lst1, lst2):
    """ Function which takes the two team's data as seperate lists and performs relevent quantile and rank calculations based on requirements """
    
    #原封不动
    Orig = ['Player', 'Winloss', 'Brownlow Votes']

    #Rank 和 Percentage皆有
    Wrangleboth = ['Kicks', 'Handballs', 'Disposals', 'Marks', 'Goals', 'Behinds', 'Tackles', 'Hitouts', 'Goal Assists', 'Inside 50s', 
                   'Clearances', 'Clangers', 'Rebound 50s', 'Frees For', 'Contested Possessions', 'Uncontested Possessions', 
                   'Effective Disposals', 'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces', 'Centre Clearances', 
                   'Stoppage Clearances', 'Score Involvements', 'Metres Gained', 'Intercepts', 'Tackles Inside 50', 'Time On Ground %']
    
    #Rank inv and percentage normal
    Wranglebothinv = ['Frees Agains', 'Turnovers']
    
    #Rank Only
    Wranglerankonly = ['Disposal Efficiency %']
    
    out1 = list()
    out2 = list()
    
    for i in range(len(lst1)):
        if lst1[i][0] in Orig:
            out1.append(lst1[i])
            out2.append(lst2[i])
            
        elif lst1[i][0] in Wrangleboth:
            out1.append(ownteamrank(lst1[i]))
            out2.append(ownteamrank(lst2[i]))
            tmp1, tmp2 = bothteamrank(lst1[i], lst2[i])
            out1.append(tmp1)
            out2.append(tmp2)
            
            out1.append(ownteamper(lst1[i]))
            out2.append(ownteamper(lst2[i]))
            tmp1, tmp2 = bothteamper(lst1[i], lst2[i])
            out1.append(tmp1)
            out2.append(tmp2)
            
        elif lst1[i][0] in Wranglebothinv:
            out1.append(ownteamrank_inv(lst1[i]))
            out2.append(ownteamrank_inv(lst2[i]))
            tmp1, tmp2 = bothteamrank_inv(lst1[i], lst2[i])
            out1.append(tmp1)
            out2.append(tmp2)
            
            out1.append(ownteamper(lst1[i]))
            out2.append(ownteamper(lst2[i]))
            tmp1, tmp2 = bothteamper(lst1[i], lst2[i])
            out1.append(tmp1)
            out2.append(tmp2)
            
        elif lst1[i][0] in Wranglerankonly:
            out1.append(ownteamrank(lst1[i]))
            out2.append(ownteamrank(lst2[i]))
            tmp1, tmp2 = bothteamrank(lst1[i], lst2[i])
            out1.append(tmp1)
            out2.append(tmp2)
            
    return out1, out2 

## 5. Crawl and Scrape ##

Crawling the urls which contain the player data of each game

In [21]:
base_url = 'https://www.footywire.com/afl/footy/'
years = range(2015, 2020)

# years = list(range(2015,2022)) # the years we are interested in
urllist = list()

for year in years:
    u = f'https://www.footywire.com/afl/footy/ft_match_list?year={year}'
    page = requests.get(u)
    soup = BeautifulSoup(page.text, 'html.parser')
    sections = soup.find_all('a')
    gameurl = list()
    for section in sections:
        if re.search(r'>\d*-\d*<', str(section)):
            gameurl.append(urljoin(base_url, section['href']))
    urllist.append(gameurl)
    
    time.sleep(random.uniform(0.5, 5))

Scraping every game in the given years using a loop.

*(This is like the overall 'Main' function in this entire notebook)*

In [22]:
count = 0
for year in urllist:
    for game in year:
        test1, test2, metadata = scrape(game)
        
        if test1 != False and test2 != False and metadata != False:
            out = combine(test1, test2)
            save(out, metadata, 'O')

            SOut1, SOut2 = standardise(test1, test2)
            SOut = combine(SOut1, SOut2)
            save(SOut, metadata, 'S')

            PROut1, PROut2 = PercentageAndRank(test1, test2)
            PROut = combine(PROut1, PROut2)
            save(PROut, metadata, 'M')
        
        time.sleep(random.uniform(0.5, 5))

2015 Round 3 Hawthorn v WesternBulldogs: 8
{'L Hodge': 3, 'J Roughead': 2, 'S Mitchell': 1}


2016 Round 9 GoldCoast v Adelaide: 8
{'T Walker': 3, 'T Lynch': 2, 'D Talia': 1}


2017 Round 4 WestCoast v Sydney: 8
{'L Shuey': 3, 'J Kennedy': 2, 'J Cripps': 1}


2017 Round 23 Hawthorn v WesternBulldogs: 9
{'J Roughead': 3, 'J Macrae': 2, "J O'Meara": 1}


2018 Round 16 Melbourne v Fremantle: 9
{'A Brayshaw': 3, 'M Gawn': 2, 'C Oliver': 1}


2019 Round 12 Adelaide v GWS: 8
{'T Taranto': 3, 'J Kelly': 2, 'B Crouch': 1}




## Note: A few improvements could be made on this notebook: ##

*1. The use of the word percentage could be replaced with quantile for better depiction of what is actually done*

*2. The crawler lacks a list to contain visited sites, and the most glaring implication is that if the looped scraping fails at a certain game, all previous games (at least in that year) needs to be re-scraped*